In [18]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
tickers = [
    'AAL', 'AAPL', 'ALNY', 'AMD', 'AMZN', 'AVPT', 'BA', 'BBIO', 'BLD', 'BYND', 
    'C', 'CFLT',  'CHWY', 'CLF', 'COST', 'DASH', 'DDD', 'DG', 'DIS', 'DKNG', 'EBS', 'EMXC', 'ESTC', 'FIS',
    'FLUT', 'GE', 'GOOGL',  'HD', 'INTC', 'ISRG', 'ITRG', 'JNJ', 'JPM', 'KHC', 'KO', 'KR', 'LGMK', 'LMT', 
    'LOCO', 'LSCC', 'LUNR', 'MDB', 'META',  'MSFT', 'MSTR', 'NFLX', 'NKE', 'NVAX', 'NVDA', 'PEP', 'PFE', 'PGR',
    'PLUG', 'PTON', 'PYPL', 'QCOM', 'RNLX', 'ROKU',  'SBUX', 'SCHR', 'SPG', 'SPOT', 'SPWR', 'SQ', 'SWKS', 
    'T', 'TGT', 'TPR', 'TSLA', 'ULTA', 'V', 'VZ', 'WFC',  'WMT', 'XOM', 'ZM', 'DIS', 'ABNB','CRWD', 'WBD', 'TMUS'
]

markets = [
    '^IXIC', '^NYA', '^GSPC'
]

In [20]:
data = yf.download(tickers, start="2021-01-01", end="2024-01-01", group_by="ticker")

# Extract the close prices
t_close = pd.DataFrame({ticker: data[ticker]['Close'] for ticker in tickers})

data = yf.download(markets, start="2021-01-01", end="2024-01-01", group_by="ticker")

# Extract the close prices
m_close = pd.DataFrame({ticker: data[ticker]['Close'] for ticker in markets})

data = yf.download(tickers, start="2021-01-01", end="2024-01-01", group_by="ticker", interval= '1mo')

# Extract the close prices
t_close_2 = pd.DataFrame({ticker: data[ticker]['Close'] for ticker in tickers})

data = yf.download(markets, start="2021-01-01", end="2024-01-01", group_by="ticker", interval = '1mo')

# Extract the close prices
m_close_2 = pd.DataFrame({ticker: data[ticker]['Close'] for ticker in markets})

[*********************100%%**********************]  80 of 80 completed
[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  80 of 80 completed
[*********************100%%**********************]  3 of 3 completed


In [21]:
t_close
print(len(t_close))
t_change = t_close.diff()
t_change = t_change.drop(t_close.index[0])
t_change = t_change.fillna(0)
#print(t_change.isna().sum())
len(t_change)

753


752

In [22]:
t_close.to_csv('Stocks Dataset.csv', index = False)
m_close.to_csv('Market Dataset.csv' , index = False)

In [23]:
m_change_2 = m_close_2.diff()
m_change_2 = m_change_2.dropna()
len(m_change_2)

35

In [24]:
t_col = [i for i in t_close.columns if i != 'Date']
m_col = [i for i in m_close.columns if i != 'Date']

In [25]:
m_dic = {}
for i in m_col: 
    mean_r = m_change_2[i].mean()
    std2 = m_change_2[i].var()
    lp = m_close_2[i].iloc[-1]
    m_dic[i] = [
        float(lp), float(mean_r), float(std2)
    ]

In [26]:
t_dic = {ticker: [] for ticker in tickers}
t_dic_df = {
    'ticker': [],               # List of stock tickers
    'latest_price': [],          # Latest price of the stock
    'expected_return': [],       # Expected return of the stock
    'daily_variance': [],        # Daily variance of the stock returns
    'monthly_beta_nasdaq': [],    # Annual beta with respect to NASDAQ
    'monthly_beta_nyse': [],      # Annual beta with respect to NYSE
    'monthly_beta_snp': [],       # Annual beta with respect to S&P
    'monthly_sharpe': [],         # Annual Sharpe ratio of the stock
    'treynor_nasdaq': [],        # Treynor ratio with respect to NASDAQ beta
    'treynor_nyse': [],          # Treynor ratio with respect to NYSE beta
    'treynor_snp': [],           # Treynor ratio with respect to S&P beta
    'monthly_var' : []
}

In [27]:
risk_free = 0.0365 #10 year tbil rate 

In [28]:
for i in t_col:
    tick = i
    mean_r = t_change[i].mean()
    std2 = t_change[i].var()
    dev = t_change[i].std()
    lp = t_close[i].iloc[-1]

    t_dic[i].append(lp)
    t_dic[i].append(mean_r)
    t_dic[i].append(std2)

    t_dic_df['ticker'].append(tick)
    t_dic_df['latest_price'].append(lp)
    t_dic_df['expected_return'].append(mean_r)
    t_dic_df['daily_variance'].append(std2)

In [29]:
print(len(t_close_2))
t_change_2 = t_close_2.diff()
t_change_2 = t_close_2.drop(t_close_2.index[0])
t_change_2 = t_change_2.fillna(0)
#print(t_change.isna().sum())
len(t_change_2)

36


35

In [30]:
for i in t_col:
    mean_r = t_change_2[i].mean()
    std2 = t_change_2[i].var()
    dev = t_change_2[i].std()
    
    nasdaq_m_sd2 = m_dic['^IXIC'][2]
    nyse_m_sd2 = m_dic['^NYA'][2]
    snp_m_sd2 = m_dic['^GSPC'][2]

    covar_nasdap = np.cov(m_change_2['^IXIC'],t_change_2[i])
    covar_nyse = np.cov(m_change_2['^NYA'], t_change_2[i])
    covar_snp = np.cov(m_change_2['^GSPC'], t_change_2[i])

    beta_nasdap = (covar_nasdap[0][1]) / nasdaq_m_sd2
    beta_nyse = (covar_nyse[0][1]) / nyse_m_sd2
    beta_snp = (covar_snp[0][1]) / snp_m_sd2

    sharpe_ratio = (mean_r - risk_free) / dev

    treynor_nasdap = (mean_r - risk_free) / beta_nasdap
    treynor_nyse = (mean_r - risk_free) / beta_nyse
    treynor_snp = (mean_r - risk_free) / beta_snp

    t_dic[i].append(beta_nasdap)
    t_dic[i].append(beta_nyse)
    t_dic[i].append(beta_snp)
    t_dic[i].append(sharpe_ratio)
    t_dic[i].append(treynor_nasdap)
    t_dic[i].append(treynor_nyse)
    t_dic[i].append(treynor_snp)
    t_dic[i].append(std2)

    # Append the monthly betas
    t_dic_df['monthly_beta_nasdaq'].append(beta_nasdap)
    t_dic_df['monthly_beta_nyse'].append(beta_nyse)
    t_dic_df['monthly_beta_snp'].append(beta_snp)
    
    # Append the Sharpe ratio
    t_dic_df['monthly_sharpe'].append(sharpe_ratio)
    
    # Append the Treynor ratios
    t_dic_df['treynor_nasdaq'].append(treynor_nasdap)
    t_dic_df['treynor_nyse'].append(treynor_nyse)
    t_dic_df['treynor_snp'].append(treynor_snp)
    t_dic_df['monthly_var'].append(std2)

In [31]:
s_df = pd.DataFrame(t_dic_df)

In [32]:
s_df.to_csv('CAPM stock.csv', index = False)

In [33]:
s_dic = pd.DataFrame(t_dic)

In [34]:
s_dic.to_csv('CAPM dic.csv', index = False)